In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_excel("../data/Train.xlsx")

In [3]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [5]:
## Prepare proposal document and final presentation.

In [6]:
# Question:
## 5w and h ## 
# 1. Which airline is most preffered airline.
# 2. Mean/Median price for which airline is highest and lowest.
# 3. For each flight which is the most common source and destination airport.
# 4. How many flights are going from each source airports and destination airport.
# 5. Which airline is having most non-stop flight.
# 6. Which day of week we have maximum and minimum flights?
# 7. During what time day we have more flights?
# 8. What is the average price for flights on each day of week?
#      a. Different hours of day [morning/afternoon/evening/night] what is the average price?
# 9. How price and duration of flight is varring?
# 10. For which source and destiantion you have maximum and minimum duration?

# Sanity Check

In [7]:
# check for duplicate data
# Date of Jorney is object need to be converted into Datetime.
# Dep_Time is fine.
# Arrival Time is incorrectly placed so need to fixed.
# Duration is available in HH:MM format need to be either in minutes or hours. checked
# Total_Stops should be numerical but right it is object.
# Check if Additional_Info column provides any meaning information or not.

In [8]:
len(df),len(df.drop_duplicates())

(10683, 10463)

In [9]:
10682 - 10460

222

In [10]:
df[df.duplicated()].head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
683,Jet Airways,1/06/2019,Delhi,Cochin,DEL → NAG → BOM → COK,14:35,04:25 02 Jun,13h 50m,2 stops,No info,13376
1061,Air India,21/05/2019,Delhi,Cochin,DEL → GOI → BOM → COK,22:00,19:15 22 May,21h 15m,2 stops,No info,10231
1348,Air India,18/05/2019,Delhi,Cochin,DEL → HYD → BOM → COK,17:15,19:15 19 May,26h,2 stops,No info,12392
1418,Jet Airways,6/06/2019,Delhi,Cochin,DEL → JAI → BOM → COK,05:30,04:25 07 Jun,22h 55m,2 stops,In-flight meal not included,10368
1674,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,18:25,21:20,2h 55m,non-stop,No info,7303


In [11]:
df.drop_duplicates(inplace=True)

In [12]:
len(df)

10463

In [13]:
# # 33,9261 #2283,9389
# df.iloc[[33,9260]]
# df.iloc[[2283,9388]]

In [14]:
def time_minute(duration):
    dur = duration.split()
    min = 0
    for x in dur:
        if x[-1] == 'h':
            min = min + int(x[:-1])*60

        if x[-1] == 'm':
            min = min + int(int(x[:-1])) 
    return min

def convert_duration_to_minutes(time):
    '''
    This function converts duration in h m to minutes:
    input : hh:mm ,hh, mm
    return:
        min
    '''
    if len(time.split(' ')) >1 :
        hh,mm = time.split(' ')
        hh,mm = int(hh[:-1]),int(mm[:-1])
        duration = hh*60+mm
    else:
        if 'h' in time:
            duration = int(time[:-1])*60
        else:
            duration= int(time[:-1])
            
    return duration

In [15]:
df['DepartureDateTime'] = df['Date_of_Journey'] + " "+ df['Dep_Time']
df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)

In [16]:
df['Duration_min'] = df['Duration'].apply(lambda x: convert_duration_to_minutes(x))

In [17]:
df['Duration_timedelta'] = pd.to_timedelta(df['Duration_min'], unit='m')

In [18]:
df["ArrivalDateTime"] = df['DepartureDateTime'] + df['Duration_timedelta']

In [19]:
df.drop(columns=['Date_of_Journey','Dep_Time','Arrival_Time','Duration'],axis=1,inplace=True)

In [20]:
stops_dict = {
    'non-stop':0,
    '2 stops':2,
    '1 stop':1,
    '3 stops':3,
    '4 stops':4
}
df['Total_Stops'] = df['Total_Stops'].replace(stops_dict)

In [21]:
df['Additional_Info'].value_counts()/len(df)*100

No info                         78.208927
In-flight meal not included     18.407722
No check-in baggage included     3.039281
1 Long layover                   0.181592
Change airports                  0.066902
Business class                   0.038230
No Info                          0.028672
1 Short layover                  0.009557
Red-eye flight                   0.009557
2 Long layover                   0.009557
Name: Additional_Info, dtype: float64

In [22]:
# Additional Info columns doesnot have any meaningful information.
df.drop(columns=['Additional_Info'],axis=1,inplace=True)

In [23]:
df.head(2)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime
0,IndiGo,Banglore,New Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00


# Missing value Handling

In [24]:
df.isnull().sum()

Airline               0
Source                0
Destination           0
Route                 1
Total_Stops           1
Price                 0
DepartureDateTime     0
Duration_min          0
Duration_timedelta    0
ArrivalDateTime       0
dtype: int64

In [25]:
df[df['Total_Stops'].isnull()]

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime
9039,Air India,Delhi,Cochin,NaN,NaN,7480,2019-05-06 09:45:00,1420,0 days 23:40:00,2019-05-07 09:25:00


In [26]:
# dropping Nan value as its a single observation.
df.dropna(inplace=True)

# Handling Categorical Values

In [27]:
# No order in columns
categorical_cols = ['Airline', 'Source', 'Destination', 'Route']
for col in categorical_cols:
    print("Column :",col," Unique values :",df[col].unique()," and No. of uniques : ",df[col].nunique())
    print("-"*90)

Column : Airline  Unique values : ['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']  and No. of uniques :  12
------------------------------------------------------------------------------------------
Column : Source  Unique values : ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']  and No. of uniques :  5
------------------------------------------------------------------------------------------
Column : Destination  Unique values : ['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad']  and No. of uniques :  6
------------------------------------------------------------------------------------------
Column : Route  Unique values : ['BLR → DEL' 'CCU → IXR → BBI → BLR' 'DEL → LKO → BOM → COK'
 'CCU → NAG → BLR' 'BLR → NAG → DEL' 'CCU → BLR' 'BLR → BOM → DEL'
 'DEL → BOM → COK' 'DEL → BLR → COK' 'MAA → CCU' 'CCU → BOM → BLR'
 'DEL → AMD 

In [28]:
df.head(1)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime
0,IndiGo,Banglore,New Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00


In [29]:
df['Destination'] = df['Destination'].replace({'New Delhi':'Delhi'})

In [30]:
# No order in columns
categorical_cols = ['Airline', 'Source', 'Destination', 'Route']
for col in categorical_cols:
    print("Column :",col," Unique values :",df[col].unique()," and No. of uniques : ",df[col].nunique())
    print("-"*90)

Column : Airline  Unique values : ['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']  and No. of uniques :  12
------------------------------------------------------------------------------------------
Column : Source  Unique values : ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']  and No. of uniques :  5
------------------------------------------------------------------------------------------
Column : Destination  Unique values : ['Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Hyderabad']  and No. of uniques :  5
------------------------------------------------------------------------------------------
Column : Route  Unique values : ['BLR → DEL' 'CCU → IXR → BBI → BLR' 'DEL → LKO → BOM → COK'
 'CCU → NAG → BLR' 'BLR → NAG → DEL' 'CCU → BLR' 'BLR → BOM → DEL'
 'DEL → BOM → COK' 'DEL → BLR → COK' 'MAA → CCU' 'CCU → BOM → BLR'
 'DEL → AMD → BOM → COK'

In [31]:
def frequency_encoder(df,col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          frequency encoded dictionary for columns
    """
    freq_value = df.groupby(col).size()/len(df)
    freq_dict = freq_value.to_dict()
    df["Freq_encoded_"+col] = df[col].replace(freq_dict)
    return freq_dict




In [32]:
def mean_encoder(df,col,target_col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          Mean encoded dict for column
    """
    mean_value = df.groupby(col)[target_col].mean()
    mean_dict = mean_value.to_dict()
    df["Mean_encoded_"+col] = df[col].replace(mean_dict)
    return mean_dict

In [33]:
## Label encoder for function and later usages:

## Applying Label Encoder to Categorical columns as Hit and Trail.
from sklearn.preprocessing import LabelEncoder

def label_encoder(df,col):
    """
    This function encodes a categorical column based on the basis of their order label.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          label encoded dict for column
    """
    le = LabelEncoder()
    le.fit(df[col])
    label_dict = dict(zip((le.classes_),le.transform(le.classes_)))
    df["Label_encoded_"+col] = df[col].replace(label_dict)
    return label_dict

In [34]:
## Create a function to handle categorical value
def handle_categorical_values(df,target):
    '''
      This function handles categorical value and create a dataframe.
      Input:
        df : Dataframe which require categorical value treatment
      returns :
         Dataframe with all categorical value handled.
    '''
    encoded_dict = {}
    # Getting all object columns
    object_columns = df.select_dtypes(object).columns

    ## generate frequency encoded categorical values
    frequency_encoded_dict ={} 
    for col in object_columns:
        freq_dict = frequency_encoder(df,col)
        frequency_encoded_dict[col] = freq_dict

    ## generate target mean encoded categorical values
    mean_encoded_dict ={} 
    for col in object_columns:
        mean_dict = mean_encoder(df,col,target)
        mean_encoded_dict[col] = mean_dict

    
    ## generate label encoded categorical values
    label_encoded_dict ={} 
    for col in object_columns:
        label_dict = label_encoder(df,col)
        label_encoded_dict[col] = label_dict
    
    encoded_dict["Frequency"] = frequency_encoded_dict
    encoded_dict["Mean"] = mean_encoded_dict
    encoded_dict["Label"] = label_encoded_dict

    return df, encoded_dict

In [35]:
target = 'Price'
df, encoded_dict = handle_categorical_values(df,target)

In [36]:
encoded_dict

{'Frequency': {'Airline': {'Air Asia': 0.030491301854329956,
   'Air India': 0.1619193270885108,
   'GoAir': 0.018543299560313517,
   'IndiGo': 0.1952781494934047,
   'Jet Airways': 0.35366086790288664,
   'Jet Airways Business': 0.0005735041101127892,
   'Multiple carriers': 0.1143184859491493,
   'Multiple carriers Premium economy': 0.0012425922385777097,
   'SpiceJet': 0.0779009749569872,
   'Trujet': 9.558401835213153e-05,
   'Vistara': 0.04568916077231887,
   'Vistara Premium economy': 0.0002867520550563946},
  'Source': {'Banglore': 0.20827757598929458,
   'Chennai': 0.03641751099216211,
   'Delhi': 0.41531255974001147,
   'Kolkata': 0.27337029248709616,
   'Mumbai': 0.06662206079143568},
  'Destination': {'Banglore': 0.27337029248709616,
   'Cochin': 0.41531255974001147,
   'Delhi': 0.20827757598929458,
   'Hyderabad': 0.06662206079143568,
   'Kolkata': 0.03641751099216211},
  'Route': {'BLR → AMD → DEL': 0.0017205123303383673,
   'BLR → BBI → DEL': 0.0004779200917606576,
   'BL

In [37]:
df

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,...,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Airline,Mean_encoded_Source,Mean_encoded_Destination,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route
0,IndiGo,Banglore,Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,...,0.208278,0.146817,5668.469897,8022.872877,8022.872877,5552.235677,3,0,2,18
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00,...,0.273370,0.000574,9556.608028,9143.083566,9143.083566,7369.166667,1,3,0,84
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2.0,13882,2019-06-09 09:25:00,1140,0 days 19:00:00,2019-06-10 04:25:00,...,0.415313,0.003919,11599.021081,10461.600690,10461.600690,13277.195122,4,2,1,118
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1.0,6218,2019-05-12 18:05:00,325,0 days 05:25:00,2019-05-12 23:30:00,...,0.273370,0.000860,5668.469897,9143.083566,9143.083566,6279.333333,3,3,0,91
4,IndiGo,Banglore,Delhi,BLR → NAG → DEL,1.0,13302,2019-03-01 16:50:00,285,0 days 04:45:00,2019-03-01 21:35:00,...,0.208278,0.000287,5668.469897,8022.872877,8022.872877,11313.000000,3,0,2,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,CCU → BLR,0.0,4107,2019-04-09 19:55:00,150,0 days 02:30:00,2019-04-09 22:25:00,...,0.273370,0.069203,5590.260188,9143.083566,9143.083566,4556.055249,0,3,0,64
10679,Air India,Kolkata,Banglore,CCU → BLR,0.0,4145,2019-04-27 20:45:00,155,0 days 02:35:00,2019-04-27 23:20:00,...,0.273370,0.069203,9556.608028,9143.083566,9143.083566,4556.055249,1,3,0,64
10680,Jet Airways,Banglore,Delhi,BLR → DEL,0.0,7229,2019-04-27 08:20:00,180,0 days 03:00:00,2019-04-27 11:20:00,...,0.208278,0.146817,11599.021081,8022.872877,8022.872877,5552.235677,4,0,2,18
10681,Vistara,Banglore,Delhi,BLR → DEL,0.0,12648,2019-03-01 11:30:00,160,0 days 02:40:00,2019-03-01 14:10:00,...,0.208278,0.146817,7801.355649,8022.872877,8022.872877,5552.235677,10,0,2,18


In [38]:
categorical_cols = ['Airline', 'Source', 'Destination', 'Route']
df.drop(columns=categorical_cols,inplace=True)

In [39]:
df.corr()

,Total_Stops,Price,Duration_min,Freq_encoded_Airline,Freq_encoded_Source,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Airline,Mean_encoded_Source,Mean_encoded_Destination,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route
Total_Stops,1.000000,0.602013,0.727982,0.211803,0.571402,0.571402,-0.205605,0.421327,0.559200,0.559200,0.806690,-0.194549,0.194203,-0.422656,0.441347
Price,0.602013,1.000000,0.501758,0.371620,0.350997,0.350997,0.038912,0.638674,0.359938,0.359938,0.746275,-0.036491,0.013796,-0.260751,0.154374
Duration_min,0.727982,0.501758,1.000000,0.289457,0.400039,0.400039,-0.111065,0.433989,0.414954,0.414954,0.671871,-0.153618,0.162808,-0.383773,0.273200
Freq_encoded_Airline,0.211803,0.371620,0.289457,1.000000,0.038026,0.038026,0.019636,0.581861,0.056089,0.056089,0.367567,-0.200348,0.055926,-0.107401,-0.065789
Freq_encoded_Source,0.571402,0.350997,0.400039,0.038026,1.000000,1.000000,0.303115,0.258498,0.975159,0.975159,0.470332,-0.049748,0.066914,-0.596026,0.612054
Freq_encoded_Destination,0.571402,0.350997,0.400039,0.038026,1.000000,1.000000,0.303115,0.258498,0.975159,0.975159,0.470332,-0.049748,0.066914,-0.596026,0.612054
Freq_encoded_Route,-0.205605,0.038912,-0.111065,0.019636,0.303115,0.303115,1.000000,0.158467,0.272478,0.272478,0.052142,0.194054,-0.196373,-0.001272,0.055678
Mean_encoded_Airline,0.421327,0.638674,0.433989,0.581861,0.258498,0.258498,0.158467,1.000000,0.257259,0.257259,0.572144,-0.057135,0.047805,-0.183451,0.071608
Mean_encoded_Source,0.559200,0.359938,0.414954,0.056089,0.975159,0.975159,0.272478,0.257259,1.000000,1.000000,0.482313,-0.059864,0.038329,-0.724431,0.496810
Mean_encoded_Destination,0.559200,0.359938,0.414954,0.056089,0.975159,0.975159,0.272478,0.257259,1.000000,1.000000,0.482313,-0.059864,0.038329,-0.724431,0.496810


In [40]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics

from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import SplineTransformer

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


def regression(X_train, X_test, y_train, y_test):
    ## Baseline model
    reg = linear_model.LinearRegression()
    reg.fit(X_train,y_train)
    print(metrics.r2_score(y_train,reg.predict(X_train)), metrics.r2_score(y_test,reg.predict(X_test)))
    return reg

def knearestneighbour(X_train, X_test, y_train, y_test):
    k_range = list(range(1, 30))
    params = dict(n_neighbors = k_range)
    knn_regressor = GridSearchCV(KNeighborsRegressor(), params, cv =10, scoring = 'neg_mean_squared_error')
    knn_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,knn_regressor.predict(X_train)),metrics.r2_score(y_test,knn_regressor.predict(X_test)))
    return knn_regressor

def decisiontree(X_train, X_test, y_train, y_test):
    depth  =list(range(3,30))
    param_grid =dict(max_depth =depth)
    tree =GridSearchCV(DecisionTreeRegressor(),param_grid,cv =10)
    tree.fit(X_train,y_train)
    print(metrics.r2_score(y_train,tree.predict(X_train)),metrics.r2_score(y_test,tree.predict(X_test)))
    return tree

def randomForest(X_train, X_test, y_train, y_test):
    random_regressor = RandomForestRegressor(n_estimators=500, min_samples_split=10,min_samples_leaf=2)
    random_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,random_regressor.predict(X_train)),metrics.r2_score(y_test,random_regressor.predict(X_test)))
    return random_regressor

from xgboost import XGBRegressor
def xgboost(X_train, X_test, y_train, y_test):
    xgboost_regressor = XGBRegressor()
    xgboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,xgboost_regressor.predict(X_train)),metrics.r2_score(y_test,xgboost_regressor.predict(X_test)))
    return xgboost_regressor

from catboost import  CatBoostRegressor
def catboost(X_train, X_test, y_train, y_test):
    catboost_regressor = CatBoostRegressor(verbose=False)
    catboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,catboost_regressor.predict(X_train)),metrics.r2_score(y_test,catboost_regressor.predict(X_test)))
    return catboost_regressor

from sklearn.ensemble import GradientBoostingRegressor
def gboost(X_train, X_test, y_train, y_test):
    gboost_regressor = GradientBoostingRegressor()
    gboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,gboost_regressor.predict(X_train)),metrics.r2_score(y_test,gboost_regressor.predict(X_test)))
    return gboost_regressor


def train(X,y,modelType):
    # Split your dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    model = modelType(X_train, X_test, y_train, y_test)
    return model


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [41]:
X = df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = df['Price']

model = train(X,y,regression)
model = train(X,y,knearestneighbour)
model = train(X,y,decisiontree)
model = train(X,y,randomForest)

0.6335750226191685 0.6191236554012092
0.6891587105979624 0.6326587229158694
0.6724611948562654 0.6349688466919685
0.7360266231034494 0.6423218452302983


In [42]:
from sklearn.linear_model import LinearRegression
def sklearn_vif(exogs, data):
    '''
    This function calculates variance inflation function in sklearn way. 
     It is a comparatively faster process.

    '''
    # initialize dictionaries
    vif_dict, tolerance_dict = {}, {}

    # form input data for each exogenous variable
    for exog in exogs:
        not_exog = [i for i in exogs if i != exog]
        X, y = data[not_exog], data[exog]

        # extract r-squared from the fit
        r_squared = LinearRegression().fit(X, y).score(X, y)

        # calculate VIF
        vif = 1/(1 - r_squared)
        vif_dict[exog] = vif

        # calculate tolerance
        tolerance = 1 - r_squared
        tolerance_dict[exog] = tolerance

    # return VIF DataFrame
    df_vif = pd.DataFrame({'VIF': vif_dict, 'Tolerance': tolerance_dict})

    return df_vif

df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)

C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)


In [43]:
df_vif

,VIF,Tolerance
Freq_encoded_Source,inf,0.000000
Freq_encoded_Destination,inf,0.000000
Mean_encoded_Source,inf,0.000000
Mean_encoded_Destination,inf,0.000000
Label_encoded_Destination,6.549515e+04,0.000015
Label_encoded_Source,1.463039e+04,0.000068
Label_encoded_Route,5.382753e+01,0.018578
Total_Stops,8.105551e+00,0.123372
Mean_encoded_Route,6.025714e+00,0.165955
Duration_min,2.349205e+00,0.425676


In [44]:
source_cols = [col for col in df.columns if 'Source' in col]
df.drop(columns=source_cols,inplace=True)
X = df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = df['Price']

model = train(X,y,randomForest)

0.7355923652211432 0.6428466566854469


In [45]:
df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)

In [46]:
df_vif

,VIF,Tolerance
Mean_encoded_Destination,114.774637,0.008713
Freq_encoded_Destination,104.782932,0.009544
Total_Stops,6.926954,0.144364
Label_encoded_Destination,6.243716,0.160161
Mean_encoded_Route,4.833028,0.206910
Label_encoded_Route,3.537661,0.282673
Duration_min,2.348953,0.425722
Mean_encoded_Airline,2.072776,0.482445
Freq_encoded_Route,1.940748,0.515265
Freq_encoded_Airline,1.704573,0.586657


In [47]:
df.drop(columns=['Mean_encoded_Destination'],inplace=True)

In [48]:
X = df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = df['Price']


model = train(X,y,randomForest)

0.7360305581732861 0.6423430710447132


In [49]:
df_vif= sklearn_vif(exogs=X.columns, data=df).sort_values(by='VIF',ascending=False)
df_vif

,VIF,Tolerance
Total_Stops,6.820147,0.146624
Mean_encoded_Route,4.747217,0.210650
Freq_encoded_Destination,3.927580,0.254610
Duration_min,2.344562,0.426519
Mean_encoded_Airline,2.051917,0.487349
Label_encoded_Route,2.029337,0.492772
Freq_encoded_Route,1.928397,0.518565
Label_encoded_Destination,1.830819,0.546204
Freq_encoded_Airline,1.701212,0.587816
Label_encoded_Airline,1.191270,0.839440


In [50]:
from sklearn.preprocessing import RobustScaler

X_transform = RobustScaler().fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)
model = train(X_transformed,y,randomForest)

0.7356715988399527 0.6432136457781432


In [51]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Airline,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.727982,0.211803,0.571402,-0.205605,0.421327,0.806690,-0.194549,-0.422656,0.441347,0.039017
Duration_min,0.727982,1.000000,0.289457,0.400039,-0.111065,0.433989,0.671871,-0.153618,-0.383773,0.273200,0.027030
Freq_encoded_Airline,0.211803,0.289457,1.000000,0.038026,0.019636,0.581861,0.367567,-0.200348,-0.107401,-0.065789,0.006926
Freq_encoded_Destination,0.571402,0.400039,0.038026,1.000000,0.303115,0.258498,0.470332,-0.049748,-0.596026,0.612054,0.039077
Freq_encoded_Route,-0.205605,-0.111065,0.019636,0.303115,1.000000,0.158467,0.052142,0.194054,-0.001272,0.055678,0.027073
Mean_encoded_Airline,0.421327,0.433989,0.581861,0.258498,0.158467,1.000000,0.572144,-0.057135,-0.183451,0.071608,0.049989
Mean_encoded_Route,0.806690,0.671871,0.367567,0.470332,0.052142,0.572144,1.000000,-0.058168,-0.349403,0.206860,0.054850
Label_encoded_Airline,-0.194549,-0.153618,-0.200348,-0.049748,0.194054,-0.057135,-0.058168,1.000000,0.068634,0.029636,0.011536
Label_encoded_Destination,-0.422656,-0.383773,-0.107401,-0.596026,-0.001272,-0.183451,-0.349403,0.068634,1.000000,-0.229487,-0.023831
Label_encoded_Route,0.441347,0.273200,-0.065789,0.612054,0.055678,0.071608,0.206860,0.029636,-0.229487,1.000000,0.014320


In [52]:
from sklearn.preprocessing import PowerTransformer

X_transform = PowerTransformer().fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)
model = train(X_transformed,y,randomForest)

0.7356753893846193 0.6435272363788169


In [53]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Airline,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.844850,0.226303,0.594835,-0.257056,0.466701,0.844533,-0.154681,-0.373699,0.458192,0.039913
Duration_min,0.844850,1.000000,0.291936,0.535225,-0.086790,0.524692,0.819401,-0.108921,-0.425656,0.352314,0.036260
Freq_encoded_Airline,0.226303,0.291936,1.000000,0.025808,0.029916,0.600581,0.355888,-0.081844,-0.097448,-0.060665,0.006537
Freq_encoded_Destination,0.594835,0.535225,0.025808,1.000000,0.215073,0.267096,0.490481,-0.015420,-0.328968,0.687665,0.038186
Freq_encoded_Route,-0.257056,-0.086790,0.029916,0.215073,1.000000,0.140361,0.009358,0.251402,0.087376,-0.032610,0.022537
Mean_encoded_Airline,0.466701,0.524692,0.600581,0.267096,0.140361,1.000000,0.601058,-0.014827,-0.139421,0.081027,0.045154
Mean_encoded_Route,0.844533,0.819401,0.355888,0.490481,0.009358,0.601058,1.000000,-0.017892,-0.296602,0.239712,0.054375
Label_encoded_Airline,-0.154681,-0.108921,-0.081844,-0.015420,0.251402,-0.014827,-0.017892,1.000000,0.054979,0.042487,0.013280
Label_encoded_Destination,-0.373699,-0.425656,-0.097448,-0.328968,0.087376,-0.139421,-0.296602,0.054979,1.000000,-0.191226,-0.015926
Label_encoded_Route,0.458192,0.352314,-0.060665,0.687665,-0.032610,0.081027,0.239712,0.042487,-0.191226,1.000000,0.014430


In [54]:
from sklearn.preprocessing import FunctionTransformer

X_transform = FunctionTransformer(np.log1p).fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)
model = train(X_transformed,y,randomForest)

0.7364165510948961 0.6417470703476531


In [55]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Airline,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.863786,0.232432,0.615532,-0.146651,0.490239,0.867788,-0.115757,-0.378406,0.368466,0.626180
Duration_min,0.863786,1.000000,0.288951,0.584356,-0.018656,0.544532,0.850498,-0.084262,-0.425020,0.273478,0.601974
Freq_encoded_Airline,0.232432,0.288951,1.000000,0.044400,0.019247,0.601074,0.342026,0.035451,-0.096738,-0.076925,0.366855
Freq_encoded_Destination,0.615532,0.584356,0.044400,1.000000,0.284566,0.289999,0.563480,-0.016786,-0.431073,0.502693,0.354439
Freq_encoded_Route,-0.146651,-0.018656,0.019247,0.284566,1.000000,0.163356,0.096027,0.257797,0.101184,0.048743,0.029534
Mean_encoded_Airline,0.490239,0.544532,0.601074,0.289999,0.163356,1.000000,0.609313,0.004491,-0.137142,0.040957,0.610871
Mean_encoded_Route,0.867788,0.850498,0.342026,0.563480,0.096027,0.609313,1.000000,0.013449,-0.312079,0.137305,0.734900
Label_encoded_Airline,-0.115757,-0.084262,0.035451,-0.016786,0.257797,0.004491,0.013449,1.000000,0.057037,0.030229,0.034959
Label_encoded_Destination,-0.378406,-0.425020,-0.096738,-0.431073,0.101184,-0.137142,-0.312079,0.057037,1.000000,-0.341735,-0.195564
Label_encoded_Route,0.368466,0.273478,-0.076925,0.502693,0.048743,0.040957,0.137305,0.030229,-0.341735,1.000000,0.054002


In [56]:
from sklearn.preprocessing import FunctionTransformer
def cube(X): return X**(0.333)

X_transform = FunctionTransformer(cube).fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)
model = train(X_transformed,y,randomForest)

0.7353831973138352 0.6424082182360651


In [57]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Airline,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.833065,0.239045,0.621873,-0.232358,0.495621,0.873024,-0.043271,-0.273600,0.420080,0.628457
Duration_min,0.833065,1.000000,0.286370,0.526672,-0.176549,0.527382,0.799758,-0.067232,-0.294052,0.297467,0.576777
Freq_encoded_Airline,0.239045,0.286370,1.000000,0.063097,0.027772,0.562445,0.328025,0.077002,-0.072750,-0.035205,0.333373
Freq_encoded_Destination,0.621873,0.526672,0.063097,1.000000,0.142225,0.290668,0.541300,-0.036344,-0.286197,0.418825,0.358264
Freq_encoded_Route,-0.232358,-0.176549,0.027772,0.142225,1.000000,0.107632,-0.058498,0.261455,0.135966,-0.040536,-0.067720
Mean_encoded_Airline,0.495621,0.527382,0.562445,0.290668,0.107632,1.000000,0.607769,0.059308,-0.069334,0.060645,0.621726
Mean_encoded_Route,0.873024,0.799758,0.328025,0.541300,-0.058498,0.607769,1.000000,0.034782,-0.182592,0.175625,0.740985
Label_encoded_Airline,-0.043271,-0.067232,0.077002,-0.036344,0.261455,0.059308,0.034782,1.000000,0.065459,0.039893,0.054084
Label_encoded_Destination,-0.273600,-0.294052,-0.072750,-0.286197,0.135966,-0.069334,-0.182592,0.065459,1.000000,-0.200193,-0.118911
Label_encoded_Route,0.420080,0.297467,-0.035205,0.418825,-0.040536,0.060645,0.175625,0.039893,-0.200193,1.000000,0.097279


In [58]:
model = train(X,y,xgboost)
model = train(X,y,catboost)
model = train(X,y,gboost)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.751058049628192 0.6296232654999112
0.7292652918747647 0.6476710811482218
0.6835326145533955 0.6476052915496514


In [59]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, include_bias=False,interaction_only=False)

In [60]:
X_transform = poly.fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=poly.get_feature_names(X.columns))
model = train(X_transformed,y,randomForest)
model = train(X_transformed,y,xgboost)
model = train(X_transformed,y,catboost)
model = train(X_transformed,y,gboost)

C:\Users\theve\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0.7420504922614536 0.6437786939629352


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7558889585669635 0.6192899099734246
0.7414142539971079 0.6437212606732532
0.7041741158282632 0.65337449228967


In [61]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Airline,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Destination,Label_encoded_Route,...,Label_encoded_Airline^2 Label_encoded_Destination,Label_encoded_Airline^2 Label_encoded_Route,Label_encoded_Airline Label_encoded_Destination^2,Label_encoded_Airline Label_encoded_Destination Label_encoded_Route,Label_encoded_Airline Label_encoded_Route^2,Label_encoded_Destination^3,Label_encoded_Destination^2 Label_encoded_Route,Label_encoded_Destination Label_encoded_Route^2,Label_encoded_Route^3,Price
Total_Stops,1.000000,0.727982,0.211803,0.571402,-0.205605,0.421327,0.806690,-0.194549,-0.422656,0.441347,...,-0.306584,-0.028720,-0.385577,-0.164184,0.165896,-0.420359,-0.275403,-0.050335,0.378800,0.039017
Duration_min,0.727982,1.000000,0.289457,0.400039,-0.111065,0.433989,0.671871,-0.153618,-0.383773,0.273200,...,-0.270878,-0.058625,-0.321736,-0.190698,0.041844,-0.346999,-0.242886,-0.095936,0.204657,0.027030
Freq_encoded_Airline,0.211803,0.289457,1.000000,0.038026,0.019636,0.581861,0.367567,-0.200348,-0.107401,-0.065789,...,-0.303175,-0.358508,-0.191894,-0.207421,-0.176365,-0.118523,-0.137826,-0.136830,-0.088243,0.006926
Freq_encoded_Destination,0.571402,0.400039,0.038026,1.000000,0.303115,0.258498,0.470332,-0.049748,-0.596026,0.612054,...,-0.371225,0.105170,-0.594323,-0.227965,0.340714,-0.727703,-0.491623,-0.112207,0.530381,0.039077
Freq_encoded_Route,-0.205605,-0.111065,0.019636,0.303115,1.000000,0.158467,0.052142,0.194054,-0.001272,0.055678,...,0.026109,0.099583,-0.095006,0.021287,0.175755,-0.181197,-0.140140,-0.042618,0.041758,0.027073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Label_encoded_Destination^3,-0.420359,-0.346999,-0.118523,-0.727703,-0.181197,-0.251697,-0.398576,0.102102,0.830037,-0.019315,...,0.548979,0.175362,0.824066,0.712434,0.178680,1.000000,0.938533,0.747125,0.154413,-0.033157
Label_encoded_Destination^2 Label_encoded_Route,-0.275403,-0.242886,-0.137826,-0.491623,-0.140140,-0.226060,-0.303257,0.112474,0.683151,0.278695,...,0.504613,0.299319,0.766083,0.807238,0.398223,0.938533,1.000000,0.918189,0.438423,-0.027101
Label_encoded_Destination Label_encoded_Route^2,-0.050335,-0.095936,-0.136830,-0.112207,-0.042618,-0.141272,-0.129033,0.102435,0.534772,0.582525,...,0.415974,0.380749,0.612491,0.818663,0.604022,0.747125,0.918189,1.000000,0.747939,-0.013679
Label_encoded_Route^3,0.378800,0.204657,-0.088243,0.530381,0.041758,0.034242,0.180877,0.039572,0.039310,0.926228,...,0.089562,0.391467,0.118689,0.520478,0.730249,0.154413,0.438423,0.747939,1.000000,0.010063


## Adding Basic Time based Features

## I

In [62]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

# 3-8 early_morning
# 8-12 morning
# 12-16 afternoon
# 16-20 evening
# 20-00 night
# 00-3 late_night

df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

In [63]:
X = df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = df['Price']

In [64]:
print(X.columns)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Freq_encoded_Destination', 'Freq_encoded_Route',
       'Mean_encoded_Airline', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded'],
      dtype='object')


In [65]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.7679953868206838 0.6386677753343406


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.666327
5,Mean_encoded_Airline,0.094957
1,Duration_min,0.089920
2,Freq_encoded_Airline,0.048295
10,dep_hr,0.029589
11,arr_hr,0.028483
13,arrival_timeOfDay_encoded,0.010962
12,departure_timeOfDay_encoded,0.008678
4,Freq_encoded_Route,0.007915
7,Label_encoded_Airline,0.005670


In [66]:
model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.705095614798555 0.6412056004277812


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.735448
5,Mean_encoded_Airline,0.153190
1,Duration_min,0.036775
2,Freq_encoded_Airline,0.022076
11,arr_hr,0.016982
10,dep_hr,0.009429
13,arrival_timeOfDay_encoded,0.009050
4,Freq_encoded_Route,0.007774
7,Label_encoded_Airline,0.002366
9,Label_encoded_Route,0.002268


In [67]:
model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')
pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7911739598784131 0.6131836750453157


,weight,gain,total_gain,cover,total_cover
Total_Stops,117.0,6927044.5,8.104642e+08,141.376068,16541.0
Duration_min,1665.0,11333157.0,1.886971e+10,1188.499756,1978852.0
Freq_encoded_Airline,245.0,81285944.0,1.991506e+10,896.469360,219635.0
Freq_encoded_Destination,73.0,4434839.0,3.237433e+08,311.273987,22723.0
Freq_encoded_Route,258.0,8981222.0,2.317155e+09,1076.844971,277826.0
Mean_encoded_Airline,98.0,138152528.0,1.353895e+10,1650.408203,161740.0
Mean_encoded_Route,219.0,720710848.0,1.578357e+11,1603.077637,351074.0
Label_encoded_Airline,62.0,13192606.0,8.179416e+08,1307.774170,81082.0
Label_encoded_Destination,23.0,6634640.5,1.525967e+08,1085.086914,24957.0
Label_encoded_Route,144.0,7092760.5,1.021358e+09,1271.423584,183085.0


In [68]:
model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

0.7671194384975737 0.6388525719291527


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,33.641783
1,Duration_min,13.085347
5,Mean_encoded_Airline,12.742877
2,Freq_encoded_Airline,7.795937
11,arr_hr,6.705330
4,Freq_encoded_Route,6.161263
10,dep_hr,4.601998
9,Label_encoded_Route,4.564760
13,arrival_timeOfDay_encoded,2.583292
0,Total_Stops,2.308554


In [69]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.7688470777558782 0.6379812762696564


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.662675
5,Mean_encoded_Airline,0.098014
1,Duration_min,0.091279
2,Freq_encoded_Airline,0.047924
10,dep_hr,0.030227
11,arr_hr,0.028571
13,arrival_timeOfDay_encoded,0.010003
12,departure_timeOfDay_encoded,0.008393
4,Freq_encoded_Route,0.008303
9,Label_encoded_Route,0.005430


## II

In [70]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

# 3-8 early_morning
# 8-12 morning
# 12-16 afternoon
# 16-20 evening
# 20-00 night
# 00-3 late_night
df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])


one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
new_df = pd.concat([df,df_oneHotEncoded],axis=1)
new_df.drop(columns=one_hot_cols,inplace=True)

In [71]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

In [72]:
print(X.columns)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Freq_encoded_Destination', 'Freq_encoded_Route',
       'Mean_encoded_Airline', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'arrival_timeOfDay_night'],
      dtype='object')


In [73]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.768504682396755 0.6367466506109541


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.664917
5,Mean_encoded_Airline,0.098609
1,Duration_min,0.088920
2,Freq_encoded_Airline,0.042451
10,dep_hr,0.025397
11,arr_hr,0.025385
4,Freq_encoded_Route,0.008264
13,arrival_timeOfDay_encoded,0.005864
9,Label_encoded_Route,0.005413
7,Label_encoded_Airline,0.005109


In [74]:
model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.7049226109301652 0.6418864737257297


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.730000
5,Mean_encoded_Airline,0.159146
1,Duration_min,0.036913
2,Freq_encoded_Airline,0.017962
11,arr_hr,0.017849
4,Freq_encoded_Route,0.008068
10,dep_hr,0.007932
13,arrival_timeOfDay_encoded,0.003811
21,arrival_timeOfDay_early_morning,0.003738
7,Label_encoded_Airline,0.003647


In [75]:
model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7907441909318923 0.6065759476437993


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,weight,gain,total_gain,cover,total_cover
Total_Stops,125.0,6.491099e+06,8.113874e+08,148.311996,18539.0
Duration_min,1544.0,1.198381e+07,1.850301e+10,1245.745483,1923431.0
Freq_encoded_Airline,232.0,8.695726e+07,2.017408e+10,1180.547363,273887.0
Freq_encoded_Destination,74.0,6.490674e+06,4.803098e+08,145.310806,10753.0
Freq_encoded_Route,232.0,9.042155e+06,2.097780e+09,1181.551758,274120.0
Mean_encoded_Airline,86.0,1.518578e+08,1.305977e+10,2135.627930,183664.0
Mean_encoded_Route,189.0,8.354491e+08,1.578999e+11,1771.883545,334886.0
Label_encoded_Airline,51.0,1.658014e+07,8.455872e+08,1385.607788,70666.0
Label_encoded_Destination,16.0,3.449516e+06,5.519226e+07,76.875000,1230.0
Label_encoded_Route,159.0,7.166880e+06,1.139534e+09,1401.754761,222879.0


In [76]:
model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

0.7682846560978025 0.6367455730983729


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,37.335564
1,Duration_min,12.809366
5,Mean_encoded_Airline,12.180031
2,Freq_encoded_Airline,6.541156
11,arr_hr,5.750235
4,Freq_encoded_Route,5.511799
10,dep_hr,3.940768
9,Label_encoded_Route,2.638977
7,Label_encoded_Airline,2.310331
8,Label_encoded_Destination,1.949076


In [77]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.7682193658430514 0.6378804837527027


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.664454
5,Mean_encoded_Airline,0.099605
1,Duration_min,0.087878
2,Freq_encoded_Airline,0.044226
10,dep_hr,0.025447
11,arr_hr,0.024873
4,Freq_encoded_Route,0.008098
13,arrival_timeOfDay_encoded,0.006445
7,Label_encoded_Airline,0.005310
9,Label_encoded_Route,0.005199


## III

In [78]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

# 3-8 early_morning
# 8-12 morning
# 12-16 afternoon
# 16-20 evening
# 20-00 night
# 00-3 late_night

df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
new_df = pd.concat([df,df_oneHotEncoded],axis=1)
new_df.drop(columns=one_hot_cols,inplace=True)

In [79]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

In [80]:
print(X.columns)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Freq_encoded_Destination', 'Freq_encoded_Route',
       'Mean_encoded_Airline', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'arrival_timeOfDay_night'],
      dtype='object')


In [81]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.8355717920413599 0.6954511997160528


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.618998
5,Mean_encoded_Airline,0.086152
1,Duration_min,0.070474
14,dep_day_of_week,0.051294
2,Freq_encoded_Airline,0.045812
10,dep_hr,0.022734
11,arr_hr,0.020766
15,arr_day_of_week,0.010954
16,dep_weekday,0.010843
4,Freq_encoded_Route,0.009440


In [82]:
model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.7389874971460669 0.6789172450082406


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.702413
5,Mean_encoded_Airline,0.140645
14,dep_day_of_week,0.043225
1,Duration_min,0.027491
2,Freq_encoded_Airline,0.027180
11,arr_hr,0.012572
9,Label_encoded_Route,0.009969
4,Freq_encoded_Route,0.008897
16,dep_weekday,0.007575
10,dep_hr,0.006980


In [83]:
model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8755851909701369 0.7035976022283565


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,weight,gain,total_gain,cover,total_cover
Total_Stops,104.0,13845911.0,1.439975e+09,258.259613,26859.0
Duration_min,1219.0,14592531.0,1.778830e+10,1250.506958,1524368.0
Freq_encoded_Airline,224.0,87263280.0,1.954697e+10,1184.330322,265290.0
Freq_encoded_Destination,59.0,19764078.0,1.166081e+09,240.932205,14215.0
Freq_encoded_Route,280.0,13434582.0,3.761683e+09,1061.371460,297184.0
Mean_encoded_Airline,104.0,131822376.0,1.370953e+10,1918.028809,199475.0
Mean_encoded_Route,249.0,638082304.0,1.588825e+11,1436.943726,357799.0
Label_encoded_Airline,70.0,13640507.0,9.548355e+08,1106.285767,77440.0
Label_encoded_Destination,16.0,9879319.0,1.580691e+08,364.125000,5826.0
Label_encoded_Route,204.0,9159186.0,1.868474e+09,1004.921570,205004.0


In [84]:
model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

0.8386426134115561 0.7177296217797855


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,32.799814
5,Mean_encoded_Airline,10.488620
14,dep_day_of_week,9.889678
1,Duration_min,8.050665
2,Freq_encoded_Airline,5.345923
4,Freq_encoded_Route,4.791262
9,Label_encoded_Route,4.170711
11,arr_hr,3.986791
15,arr_day_of_week,3.183384
10,dep_hr,2.502915


In [85]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.836362168558915 0.6964629850791275


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.619604
5,Mean_encoded_Airline,0.088609
1,Duration_min,0.070686
14,dep_day_of_week,0.051072
2,Freq_encoded_Airline,0.041936
10,dep_hr,0.023690
11,arr_hr,0.020233
15,arr_day_of_week,0.011198
16,dep_weekday,0.010327
4,Freq_encoded_Route,0.009255


## IV

In [86]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)
# 3-8 early_morning
# 8-12 morning
# 12-16 afternoon
# 16-20 evening
# 20-00 night
# 00-3 late_night
df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['dep_month'] = df['DepartureDateTime'].dt.month
df['dep_day_of_month'] = df['DepartureDateTime'].dt.day

df['arr_month'] = df['ArrivalDateTime'].dt.month
df['arr_day_of_month'] = df['ArrivalDateTime'].dt.day

df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
new_df = pd.concat([df,df_oneHotEncoded],axis=1)
new_df.drop(columns=one_hot_cols,inplace=True)

In [87]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

In [88]:
print(X.columns)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Freq_encoded_Destination', 'Freq_encoded_Route',
       'Mean_encoded_Airline', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'dep_month', 'dep_day_of_month', 'arr_month', 'arr_day_of_month',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'arrival_timeOfDay_night'],
      dtype='object')


In [89]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.9127797955683054 0.8346214215232145


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.581559
5,Mean_encoded_Airline,0.081765
1,Duration_min,0.050595
21,arr_day_of_month,0.046762
19,dep_day_of_month,0.046578
2,Freq_encoded_Airline,0.039907
20,arr_month,0.024349
18,dep_month,0.024321
9,Label_encoded_Route,0.019157
10,dep_hr,0.014184


In [90]:
model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.8248432020119472 0.7991555909469367


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,0.605505
5,Mean_encoded_Airline,0.142204
21,arr_day_of_month,0.049687
19,dep_day_of_month,0.038723
18,dep_month,0.034773
20,arr_month,0.029575
1,Duration_min,0.022807
2,Freq_encoded_Airline,0.020321
11,arr_hr,0.015049
4,Freq_encoded_Route,0.009586


In [91]:
model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9482279958647071 0.8403034607523993


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,weight,gain,total_gain,cover,total_cover
Total_Stops,84.0,1.457020e+07,1.223897e+09,150.285721,12624.0
Duration_min,1044.0,1.412395e+07,1.474541e+10,1152.563232,1203276.0
Freq_encoded_Airline,235.0,7.292378e+07,1.713709e+10,1011.757446,237763.0
Freq_encoded_Destination,49.0,1.212284e+07,5.940194e+08,307.510193,15068.0
Freq_encoded_Route,247.0,1.613929e+07,3.986406e+09,1131.813721,279558.0
Mean_encoded_Airline,118.0,1.221187e+08,1.441001e+10,1564.000000,184552.0
Mean_encoded_Route,238.0,6.681371e+08,1.590166e+11,1414.424316,336633.0
Label_encoded_Airline,74.0,2.009185e+07,1.486797e+09,940.148621,69571.0
Label_encoded_Destination,20.0,1.188068e+07,2.376137e+08,791.000000,15820.0
Label_encoded_Route,205.0,2.940696e+07,6.028428e+09,1252.204834,256702.0


In [92]:
model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

0.9236163885562084 0.8481343501262277


,feature_names_in_,feature_importances_
6,Mean_encoded_Route,27.054033
5,Mean_encoded_Airline,9.316487
18,dep_month,8.437596
19,dep_day_of_month,8.225847
21,arr_day_of_month,7.675908
20,arr_month,6.956852
1,Duration_min,6.843646
2,Freq_encoded_Airline,4.269061
4,Freq_encoded_Route,3.298229
9,Label_encoded_Route,2.931280


In [104]:
df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)

C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_18428/2747831194.py:20: Ru

In [105]:
df_vif

,VIF,Tolerance
arrival_timeOfDay_night,inf,0.000
dep_month,inf,0.000
arrival_timeOfDay_morning,inf,0.000
arrival_timeOfDay_late_night,inf,0.000
arrival_timeOfDay_evening,inf,0.000
arrival_timeOfDay_early_morning,inf,0.000
arrival_timeOfDay_after_noon,inf,0.000
departure_timeOfDay_night,inf,0.000
departure_timeOfDay_morning,inf,0.000
departure_timeOfDay_late_night,inf,0.000


In [96]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [98]:
X.columns

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Freq_encoded_Destination', 'Freq_encoded_Route',
       'Mean_encoded_Airline', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'dep_month', 'dep_day_of_month', 'arr_month', 'arr_day_of_month',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'arrival_timeOfDay_night'],
      dtype='object')

In [124]:
cols = ['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Freq_encoded_Destination', 'Freq_encoded_Route',
       'Mean_encoded_Airline', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
#        'departure_timeOfDay_encoded',
#         'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
#        'dep_month', 
#         'dep_day_of_month', 
        'arr_month', 
        'arr_day_of_month',
       'departure_timeOfDay_after_noon', 
        'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 
        'departure_timeOfDay_late_night',
#        'departure_timeOfDay_morning', 
#         'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 
       'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 
       'arrival_timeOfDay_late_night',
#        'arrival_timeOfDay_morning',
#        'arrival_timeOfDay_night'
       ]
df_vif= sklearn_vif(exogs=cols, data=X).sort_values(by='VIF',ascending=False)

In [125]:
df_vif

,VIF,Tolerance
dep_weekday,9.880,0.101
arr_weekday,9.068,0.110
Total_Stops,7.107,0.141
dep_day_of_week,7.069,0.141
arr_day_of_week,6.626,0.151
Mean_encoded_Route,5.022,0.199
Freq_encoded_Destination,4.088,0.245
arr_hr,3.364,0.297
arrival_timeOfDay_late_night,3.008,0.332
dep_hr,2.759,0.362


In [126]:
model = train(X[cols],y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9473302518137094 0.8383735578979831


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,weight,gain,total_gain,cover,total_cover
Total_Stops,83.000,13795997.000,1145067776.000,302.807,25133.000
Duration_min,1043.000,14648590.000,15278479360.000,1231.564,1284521.000
Freq_encoded_Airline,231.000,75625240.000,17469429760.000,927.970,214361.000
Freq_encoded_Destination,56.000,14013960.000,784781760.000,495.214,27732.000
Freq_encoded_Route,280.000,9039967.000,2531190784.000,970.036,271610.000
Mean_encoded_Airline,102.000,138984240.000,14176393216.000,1600.833,163285.000
Mean_encoded_Route,229.000,695817984.000,159342313472.000,1529.236,350195.000
Label_encoded_Airline,95.000,16050878.000,1524833408.000,1213.990,115329.000
Label_encoded_Destination,27.000,13922682.000,375912416.000,856.185,23117.000
Label_encoded_Route,179.000,35131916.000,6288612864.000,974.078,174360.000


In [186]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost
xgboost_model = xgboost.XGBRegressor()

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X[cols], y, test_size=0.33, random_state=42)

In [188]:
xgboost_model.fit(X_train,y_train)
# importance_gain = model.get_booster().get_score(importance_type='gain')
# importance_cover = model.get_booster().get_score(importance_type='cover')
# importance_weight = model.get_booster().get_score(importance_type='weight')
# importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
# importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

# pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [189]:
print(metrics.r2_score(y_train,xgboost_model.predict(X_train)),metrics.r2_score(y_test,xgboost_model.predict(X_test)))

0.9473302518137094 0.8383735578979831


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [ ]:
https://www.kaggle.com/datasets/thedevastator/now-with-more-nutrients
https://www.kaggle.com/datasets/rajanand/crime-in-india
https://www.kaggle.com/datasets/vjchoudhary7/hr-analytics-case-study
https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset
https://www.kaggle.com/datasets/shashwatwork/dataco-smart-supply-chain-for-big-data-analysis
https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis
https://www.kaggle.com/datasets/mirbektoktogaraev/should-this-loan-be-approved-or-denied
https://designmodo.com/bootstrap-5-form/